In [81]:
suppressMessages(devtools::install_github("clairbarnes/rwwa"))
suppressMessages(library(rwwa))

varnm <- "tx5x"
rnm <- "kr"
nsamp <- 1000

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# specify arguments for all models
hdwi_args <- list(dist = "gev", type = "shift", varnm = "x", covnm = c("gmst"), lower = F)
pr_args <- list(dist = "norm", type = "shift", varnm = "logx", covnm = c("gmst"), lower = T)
tx_args <- list(dist = "gev", type = "shift", varnm = "x", covnm = c("gmst"), lower = F)
wind_args <- list(dist = "gev", type = "shift", varnm = "x", covnm = c("gmst"), lower = F)
hurs_args <- list(dist = "gev", type = "shift", varnm = "x", covnm = c("gmst"), lower = T)
vpd_args <- list(dist = "gev", type = "shift", varnm = "x", covnm = c("gmst"), lower = F)

mdl_args <- switch(varnm, "hdwi" = hdwi_args, "pr-fm" = pr_args, "tx5x" = tx_args, "sfcWindmax5x" = wind_args, "vpd5x" = vpd_args)
rp <- switch(varnm, "hdwi" = 100, "tx5x" = 100, "pr-fm" = 5, "sfcWindmax5x" = 5, "vpd5x" = 50)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# function to fill infinite probability ratios
infer_infinite <- function(models_in) {
    
    # split out the columns for easier reference
    est <- est_n <- models_in[,1]
    l <- l_n <- models_in[,2]
    u <- u_n <- models_in[,3]
    
    # relabel where infinite best estimate / upper bound occurs
    mnames <- rownames(models_in)
    mnames <- paste0(mnames, c("*","")[is.finite(est)+1])
    mnames <- paste0(mnames, c("*","")[is.finite(u)+1])
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # INFINITE BEST ESTIMATES / UPPER BOUNDS
    # get upper bound where best estimate is infinite
    u_n[!is.finite(u_n)] <- (exp(log(est) + 3 * (log(est) - log(l))))[!is.finite(u_n)]

    # get best estimate, if infinite
    if(any(is.finite(u))) {
        # if any estimated upper bounds are finite, use the maximum upper bound to infer infinite best estimates
        f_upper <- max(u[is.finite(u)])
    } else {
        # otherwise, use the maximum inferred upper bound to infer infinite best estimates
        f_upper <- max(u_n[is.finite(u_n)])
    }
    est_n[!is.finite(est_n)] <- f_upper

    # repeat estimation of upper bound using inferred best estimate
    u_n[!is.finite(u_n)] <- (exp(log(est_n) + 3 * (log(est_n) - log(l))))[!is.finite(u_n)]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # INFINITE LOWER BOUNDS NOT YET IMPLEMENTED

    return(data.frame("est" = est_n, "lower" = l_n, "upper" = u_n, row.names = mnames))   
}

# Trend in obs

In [16]:
gmst <- read.table("ts-kr/gmst.txt", col.names = c("year", "gmst"))

cov_2025 <- gmst[gmst$year == 2025,"gmst",drop = F]
cov_pi <- cov_2025 - 1.3
cov_fut <- cov_2025 + 1.3

In [17]:
for(fnm in list.files("ts-kr", pattern = paste0(varnm,".+ERA5land_",rnm), full.names = T)) {
    if (varnm == "pr-fm") {mtype <- "lnorm"} else {mtype <- mdl_args$dist}
    res_fnm <- gsub(".dat", paste0("_",mtype,".csv"), gsub("ts-kr/korea-fires", "res-obs/res-fut", fnm))
    
    if(!file.exists(res_fnm)) {
        ts <- load_ts(fnm, col.names = c("year", "x"))
        df <- merge(gmst, ts)
        df$logx <- log(df$x)
                
        # get model parameters for selected model
        mdl <- do.call(fit_ns, append(mdl_args, list("data" = df)))
        boot_res <- boot_ci(mdl, cov_f = cov_2025, cov_cf = rbind("pi" = cov_pi, "fut" = cov_fut))
        write.csv(boot_res, res_fnm)
    }
}

In [82]:
# compile results
obs_res <- t(sapply(list.files("res-obs", pattern = paste0("res-fut_",varnm,".+",rnm,"_.+"), full.names = T), function(fnm) {
    res <- read.csv(fnm, row.names = "X")
    unlist(lapply(rownames(res), function(cnm) setNames(res[cnm,], paste(gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
}))

# rownames(obs_res) <- sapply(strsplit(rownames(obs_res), "_"), "[", 2)
rownames(obs_res) <- "ERA5land"
write.csv(obs_res, paste0("res-fut_",varnm,"_",rnm,".csv"))

# Synthesise future changes with models

In [101]:
varnm <- "tx5x"

stype <- switch(varnm, "tx5x" = "abs", "rel")
rnm <- "kr"

In [102]:
stitle <- paste0(switch(stype, "abs" = "Change in ", "rel" = "% change in "), toupper(varnm))

dI_xlim <- switch(paste0(varnm,stype), "hdwiabs" = c(-4,6), "hdwirel" = c(-25,50), "tx5xabs" = c(-1,7), "pr-fmrel" = c(-50,150), "pr-fmabs" = c(-25,80))
PR_xlim <- switch(varnm, "hdwi" = c(1e-3,1e5), "tx5x" = c(0.1,100000))

In [103]:
# res_obs <- read.csv(paste0("res-rp100_",varnm,"_",rnm,".csv"), row.names = "X")
res_obs <- read.csv(paste0("res-fut_",varnm,"_",rnm,".csv"), row.names = "X")

res_models <- read.csv(list.files(pattern = paste0("res-models_",varnm,".+",rnm,".csv")), row.names = "X")
res_models <- res_models[res_models[,"include"] == "Y",]

In [106]:
res_princeton <- read.csv(paste0("res-princeton_",varnm,"_kr.csv"), row.names = "X")

am25_dI_attr <- synthesis(obs_in = NA,
                          models_in = infer_infinite(res_princeton[grepl("AM2", rownames(res_princeton)), grepl(paste0("attr_dI.",stype), colnames(res_princeton))]),
                          synth_type = stype)

am25_PR_attr <- synthesis(obs_in = NA,
                          models_in = infer_infinite(res_princeton[grepl("AM2", rownames(res_princeton)), grepl("attr_PR", colnames(res_princeton))]),
                          synth_type = "PR")

am25_dI_proj <- synthesis(obs_in = NA,
                          models_in = infer_infinite(res_princeton[grepl("AM2", rownames(res_princeton)), grepl(paste0("proj_dI.",stype), colnames(res_princeton))]),
                          synth_type = stype)

am25_PR_proj <- synthesis(obs_in = NA,
                          models_in = infer_infinite(res_princeton[grepl("AM2", rownames(res_princeton)), grepl("proj_PR", colnames(res_princeton))]),
                          synth_type = "PR")

flor_dI_attr <- synthesis(obs_in = NA,
                          models_in = infer_infinite(res_princeton[grepl("FLOR", rownames(res_princeton)), grepl(paste0("attr_dI.",stype), colnames(res_princeton))]),
                          synth_type = stype)

flor_PR_attr <- synthesis(obs_in = NA,
                          models_in = infer_infinite(res_princeton[grepl("FLOR", rownames(res_princeton)), grepl("attr_PR", colnames(res_princeton))]),
                          synth_type = "PR")

flor_dI_proj <- synthesis(obs_in = NA,
                          models_in = infer_infinite(res_princeton[grepl("FLOR", rownames(res_princeton)), grepl(paste0("proj_dI.",stype), colnames(res_princeton))]),
                          synth_type = stype)

flor_PR_proj <- synthesis(obs_in = NA,
                          models_in = infer_infinite(res_princeton[grepl("FLOR", rownames(res_princeton)), grepl("proj_PR", colnames(res_princeton))]),
                          synth_type = "PR")

res_flor <- data.frame(c(setNames(flor_PR_attr$df[nrow(flor_PR_attr$df),3:5], paste0("attr_PR_",c("est", "lower", "upper"))),
                         setNames(flor_dI_attr$df[nrow(flor_dI_attr$df),3:5], paste0("attr_dI.",stype, "_",c("est", "lower", "upper"))),
                         setNames(flor_PR_proj$df[nrow(flor_PR_proj$df),3:5], paste0("proj_PR_",c("est", "lower", "upper"))),
                         setNames(flor_dI_proj$df[nrow(flor_dI_proj$df),3:5], paste0("proj_dI.",stype, "_",c("est", "lower", "upper")))), 
                       row.names = "FLOR (10)")

res_am25 <- data.frame(c(setNames(am25_PR_attr$df[nrow(am25_PR_attr$df),3:5], paste0("attr_PR_",c("est", "lower", "upper"))),
                         setNames(am25_dI_attr$df[nrow(am25_dI_attr$df),3:5], paste0("attr_dI.",stype, "_",c("est", "lower", "upper"))),
                         setNames(am25_PR_proj$df[nrow(am25_PR_proj$df),3:5], paste0("proj_PR_",c("est", "lower", "upper"))),
                         setNames(am25_dI_proj$df[nrow(am25_dI_proj$df),3:5], paste0("proj_dI.",stype, "_",c("est", "lower", "upper")))), 
                       row.names = "AM2.5C360 (3)")

res_allmodels <- rbind(res_models[,colnames(res_flor)], res_flor, res_am25)

In [107]:
synth_dI_attr <- synthesis(obs_in = res_obs[,grepl(paste0(stype, ".pi"), colnames(res_obs)),drop = F],
                           models_in = res_allmodels[,grepl(paste0("attr_dI.",stype), colnames(res_allmodels))],
                           synth_type = stype)

synth_PR_attr <- synthesis(obs_in = infer_infinite(res_obs[,grepl("PR.pi", colnames(res_obs)),drop = F]),
                           models_in = infer_infinite(res_allmodels[,grepl("attr_PR", colnames(res_allmodels))]),
                           synth_type = "PR")

synth_dI_proj <- synthesis(obs_in = res_obs[,grepl(paste0(stype, ".pi"), colnames(res_obs)),drop = F],
                           models_in = res_allmodels[,grepl(paste0("proj_dI.",stype), colnames(res_allmodels))],
                           synth_type = stype)

synth_PR_proj <- synthesis(obs_in = infer_infinite(res_obs[,grepl("PR.pi", colnames(res_obs)),drop = F]),
                           models_in = infer_infinite(res_allmodels[,grepl("proj_PR", colnames(res_allmodels))]),
                           synth_type = "PR")

# rename models with infinite values
synth_dI_proj$df$model <- synth_PR_proj$df$model 
synth_dI_attr$df$model <- synth_PR_attr$df$model

Warning message in max(u_n[is.finite(u_n)]):
“no non-missing arguments to max; returning -Inf”
Warning message in log(est_n):
“NaNs produced”
Warning message in log(est_n):
“NaNs produced”
Warning message in FUN(X[[i]], ...):
“NaNs produced”
Warning message in max(u_n[is.finite(u_n)]):
“no non-missing arguments to max; returning -Inf”
Warning message in log(est_n):
“NaNs produced”
Warning message in log(est_n):
“NaNs produced”
Warning message in FUN(X[[i]], ...):
“NaNs produced”


In [108]:
write.csv(synth_dI_attr$df, paste0("synth/synth-fut_",varnm,"_",rnm,"_dI-",stype,"_attr.csv"), row.names = F)
write.csv(synth_PR_attr$df, paste0("synth/synth-fut_",varnm,"_",rnm,"_PR_attr.csv"), row.names = F)
write.csv(synth_dI_proj$df, paste0("synth/synth-fut_",varnm,"_",rnm,"_dI-",stype,"_proj.csv"), row.names = F)
write.csv(synth_PR_proj$df, paste0("synth/synth-fut_",varnm,"_",rnm,"_PR_proj.csv"), row.names = F)

In [109]:
png(paste0("fig/synth-fut_",varnm,"_",rnm,"_attr-",stype,".png"), width = 3 * 240); {
    prep_window(c(1,2), h = 8, w = 5, oma = c(0,18,0,0), mar = c(3,0.5,2,0.5))

    plot_synthesis(synth_dI_attr, main = stitle, xlim = dI_xlim)
    plot_synthesis(synth_PR_attr, hide_labels = T, main = "Change in likelihood", xlim = PR_xlim)
}; dev.off()

png 
  2

In [110]:
png(paste0("fig/synth-fut_",varnm,"_",rnm,"_proj-",stype,".png"), width = 3 * 240); {
    prep_window(c(1,2), h = 8, oma = c(0,18,0,0), mar = c(3,0.5,2,0.5))

    plot_synthesis(synth_dI_proj, main = stitle, add_space = F, xlim = dI_xlim)
    plot_synthesis(synth_PR_proj, hide_labels = T, main = "Change in likelihood", add_space = F, xlim = PR_xlim)
}; dev.off()

png 
  2